# Coverage Analysis

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import poliastro 

import CtllDes 
from CtllDes.core import ctll, satellite


### Building test satellite

In [2]:
from poliastro.bodies import Earth

sat = satellite.Sat.from_vectors([8000,0,0]*u.km,
                       [0,5,2.5]*u.km/u.s,
                      attractor=Earth)

### Add Coverage Instrument

#### Camera symmetric FOV

In [3]:
from CtllDes.core.instrument import Instrument, Camera
cam = Camera(10,3)
sat.update_instruments(cam,f=True)

#check if Camera is a Coverage instrument, more on this later 
sat.cov_instruments

#### Push Broom Instrument

In [4]:
from CtllDes.core.instrument import PushBroom

broom = PushBroom(0.314*u.rad)
sat.update_instruments(broom,f=True)

#check if PushBroom is a Coverage instrument, more on this later 
sat.cov_instruments

# Defining targets 

In [68]:
#In order to do a coverage analysis you must have targets. The module targets is the one in charge of that.
from CtllDes.targets.targets import Targets, Target
from shapely.geometry import Point

#simple target
tgt = Target(0,0)

#multiple targets
tgts = Targets([Target(i,i) for i in range(0,180,10)],tag='linear targets')

#define targets from country, administration level 0. 
tgts = Targets.from_country('Argentina')
figc = tgts.plot()
plt.title("Argentina, N=50")
plt.grid()
plt.xlabel("longitude [°]")
plt.ylabel("latitude [°]")
plt.show()


#define targets from state name, administration level 1
tgts = Targets.from_state('Río Negro', N=100)
figs = tgts.plot()
plt.title("Río Negro, N=100")
plt.xlabel("longitude [°]")
plt.ylabel("latitude [°]")
plt.grid()

plt.show()

In [43]:
# less points for country targets
tgts = Targets.from_country('Peru', N=60)

# Building Coverages
Coverages is the main container for Coverage analysis, it consist on Coverage (singular) objects. This objects are defined by 

<ul>
    <li>covs, an array with length = T*3600*24/dt containing ones or zeroes depending if the target is on sight or not.
    <li>Targets described earlier in this notebook
    <li>T == Time of propagation analysis
    <li>dt == Time interval of integration 
    <li>Merit figures
</ul>

If you want more information on the merit figures calculated for each target, I recommend reading the chapter 9 of O.C.D.M. from James R. Wertz.

<p style="text-align:center">
    <a href="https://www.amazon.es/Constellation-Design-Management-Technology-Library/dp/1881883078">
        <img src="https://images-na.ssl-images-amazon.com/images/I/41Ca0XLUv6L._SX303_BO1,204,203,200_.jpg">
    </a>
</p>

In [7]:
from CtllDes.requests.coverage import Coverages

In [8]:
#Build Coverages from satellite and single target
covs = Coverages.from_sat(sat, tgt, 10, dt=10, J2=True, drag=False)

#transform coverages into dataframe
covs.to_df()


/home/vancii/Documents/Instituto Balseiro/6to semestre/pi/CtllDes/venv/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in arccos
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/vancii/Documents/Instituto Balseiro/6to semestre/pi/CtllDes/CtllDes/utils/trigsf.py:59: RuntimeWarning: invalid value encountered in double_scalars
  A = np.arccos((cos_a-cos_b*cos_c)/(sin_b*sin_c))
/home/vancii/Documents/Instituto Balseiro/6to semestre/pi/CtllDes/CtllDes/utils/trigsf.py:60: RuntimeWarning: invalid value encountered in double_scalars
  B = np.arccos((cos_b-cos_a*cos_c)/(sin_a*sin_c))


target 0.00° 0.00°. 1 of 1


,T,dt,Satellite ID,Target,accumulated,mean gap light,mean gap dark,response time,average time gap,max gap
0,10,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(0, 0)",40,10.0,172790.0,125407.039202,250804.078867,368430


In [9]:
from CtllDes.requests.coverage import Coverages
covs = Coverages.from_sat(sat, tgts, 10, dt=100, J2=True, drag=False)
dfcov = covs.to_df()

/home/vancii/Documents/Instituto Balseiro/6to semestre/pi/CtllDes/venv/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in arccos
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


target -73.72° -15.31°. 1 of 13
target -71.20° -15.31°. 2 of 13
target -76.24° -12.27°. 3 of 13
target -73.72° -12.27°. 4 of 13
target -71.20° -12.27°. 5 of 13
target -76.24° -9.23°. 6 of 13
target -73.72° -9.23°. 7 of 13
target -78.77° -6.19°. 8 of 13
target -76.24° -6.19°. 9 of 13
target -73.72° -6.19°. 10 of 13
target -76.24° -3.15°. 11 of 13
target -73.72° -3.15°. 12 of 13
target -71.20° -3.15°. 13 of 13


In [10]:
dfcov

,T,dt,Satellite ID,Target,accumulated,mean gap light,mean gap dark,response time,average time gap,max gap
0,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -15.30859733333331)",200,100.0,287900.0,253402.951731,506705.926612,641500
1,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-71.19706999999998, -15.30859733333331)",300,100.0,215900.0,247665.320060,495230.674847,633700
2,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -12.268649666666647)",400,100.0,172700.0,191456.777405,382813.601111,545300
3,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -12.268649666666647)",100,100.0,431900.0,266828.290311,533556.592198,641500
4,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-71.19706999999998, -12.268649666666647)",200,100.0,287900.0,212885.299224,425670.621600,558000
5,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -9.228701999999984)",300,100.0,215900.0,204830.119227,409560.273180,545200
6,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -9.228701999999984)",400,100.0,172700.0,191730.153953,383360.354208,545200
7,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-78.76656799999994, -6.188754333333321)",100,100.0,431900.0,234322.965621,468545.942817,557800
8,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -6.188754333333321)",100,100.0,431900.0,264431.022109,528762.055793,636500
9,10,100,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -6.188754333333321)",300,100.0,215900.0,190340.398194,380580.831115,540300


In [11]:
lons,lats = sat.ssps(10, dt=5, J2=True, drag=False)
lons = lons*180/np.pi
lats = lats*180/np.pi

In [38]:
%matplotlib qt5

target_lons = [tgts.targets[i].lon + 180 for i in range(len(tgts.targets))]
target_lats = [tgts.targets[i].lat for i in range(len(tgts.targets))]

plt.figure(figsize=(10,10))
plt.scatter(lons,lats,c='red',s=1)
plt.ylim(-90,90)
plt.scatter(target_lons,target_lats,c='k', s=5)

In [13]:
accum = dfcov['accumulated'].values
accum = np.array([float(i) for i in accum])
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
ax.plot_trisurf(target_lons, target_lats, accum,
               antialiased=False)
ax.set_xlim(min(target_lons),max(target_lons))
ax.set_ylim(min(target_lats),max(target_lats))
#ax.scatter(lons,lats, np.zeros(len(lats)),s=1)
ax.scatter(target_lons,target_lats, np.zeros(len(target_lons)),s=100,c='k')

# What is a Coverage Instrument?

In order to be a Coverage Instrument first of all the object must be an Instrument. The coverage ability is defined by the interface of the library, i.e. a coverage method must be overwritten. See the example below.

In [14]:
#first lets check out the coverage method requirements to be correcly overwritten.
help(Instrument.coverage)

Help on function coverage in module CtllDes.core.instrument:

coverage(self, lons, lats, r, v, target, R)
    Coverage functions is associated with the coverage module.
    Any overwrited child method coverage must accept the specified parameters
    and return a list or iterable with 1 or 0, in view or not respectively.
    
    Parameters
    ----------
    lons : ~astropy.units.quantity.Quantity 
            array of longitudes as they come from the ssps method.
    lats : ~astropy.units.quantity.Quantity 
            array of latittudes as they come from the ssps method
    r : ~astropy.units.quantity.Quantity
            satellite's positions
    v : ~astropy.units.quantity.Quantity
            satellite's velocities
    target : ~CtllDes.targets.targets.Target
            desired target of coverage analysis
    R : ~astropy.units.quantity.Quantity 
            attractor mean radius 
    
    Returns
    -------
    cov : Iterable
            elements from iterable must be 1 or 0 

In [15]:
#So if you want to build a taylor made instrument, first you must specify 
#the correct arguments to the coverage method. And most importantly, return
#an Iterable containing ones or zeroes depending on the target being seen or not
#at that r,v. 

class GodInstrument(Instrument):
    def __init__(self):
        super().__init__()

    def coverage(self, lons, lats, r, v, target, R):
        return [1 for _ in range(len(r))]
    
#as you can see this is a silly example, God sees it all.

In [16]:
from CtllDes.requests.coverage import symmetric_disk

#What does exactly symmetric_disk do?
help(symmetric_disk)



#A more realistic Instrument that uses one of the few coverage methods already written.

class DiskInstrument(Instrument):
    def __init__(self):
        super().__init__()
        self.FOV_min = 0.1*u.rad
        self.FOV_max = 0.2*u.rad
        
    def coverage(self, lons, lats, r, v, target, R):
        return coverage.symmetric_disk(self.FOV_min,
                                      self.FOV_max,
                                      lons,
                                      lats,
                                      r,
                                      v,
                                      target,
                                      R)


Help on function symmetric_disk in module CtllDes.requests.coverage:

symmetric_disk(FOV_min, FOV_max, lons, lats, r, target, R)
    coverage method.
    
    Disk of coverage centered on subsatellite point.
    
    Parameters
    ----------
    FOV_min : ~astropy.units.quantity.Quantity
            minimum field of view in radians
    FOV_max : ~astropy.units.quantity.Quantity
            maximum field of view in radians
    
    * : default coverage parameters
            help(CtllDes.request.coverage.Instrument.coverage) for more
            info.



So the intuition here you must get is that the interface is the coverage method, with the default parameters needed to compute coverage figures.If you have extra parameters that define the coverage, for example, a roll angle allowed, this must be included as a parameter of the specific Instrument child class. 


In [17]:
#Define your own parameters.

from CtllDes.utils import trigsf


class OnOffCamera(Instrument):
    def __init__(self, thresh):
        super().__init__()
        self.threshold = thresh
        self._FOV = np.pi*u.rad/8
        
    @property
    def threshold(self):
        return self._threshold
    
    @threshold.setter
    def threshold(self,thresh):
        if not isinstance(thresh,u.Quantity):
            thresh = thresh * u.km
        elif thresh.unit.physical_type != 'length':
            raise ValueError("threshold must be length quantity")
        self._threshold = thresh.to(u.km)
    
    @property
    def FOV(self):
        return self._FOV
    
    
    def coverage(self,lons,lats,r,v,target,R):
        lams = trigsf.get_lam(r,self.FOV,R)
        
        angles = trigsf.get_angles(lons,lats,(target.x*u.deg).to(u.rad),
		(target.y*u.deg).to(u.rad))
        
        radiis = np.sqrt(np.sum(r**2,axis=1))
        
        cov = []
        for lam,angle,radii in zip(lams,angles,radiis):
            if angle < lam:
                if self.threshold < radii < 2*R :
                    cov.append(1)
                else:
                    cov.append(0)
            else:
                cov.append(0)
                
        return cov


In [18]:
onoffcam = OnOffCamera(300)
sat.update_instruments(onoffcam,f=True)
sat.instruments

In [19]:
newcovs = Coverages.from_sat(sat, tgts, 5, dt=5, J2=True)

/home/vancii/Documents/Instituto Balseiro/6to semestre/pi/CtllDes/venv/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in arcsin
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


target -73.72° -15.31°. 1 of 13
target -71.20° -15.31°. 2 of 13
target -76.24° -12.27°. 3 of 13
target -73.72° -12.27°. 4 of 13
target -71.20° -12.27°. 5 of 13
target -76.24° -9.23°. 6 of 13
target -73.72° -9.23°. 7 of 13
target -78.77° -6.19°. 8 of 13
target -76.24° -6.19°. 9 of 13
target -73.72° -6.19°. 10 of 13
target -76.24° -3.15°. 11 of 13
target -73.72° -3.15°. 12 of 13
target -71.20° -3.15°. 13 of 13


More spherical trigonometry calculations will be added (in development right now) to create coverage methods easier and faster.

In [20]:
newcovs.to_df()

,T,dt,Satellite ID,Target,accumulated,mean gap light,mean gap dark,response time,average time gap,max gap
0,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -15.30859733333331)",0,0.0,432000.000000,216002.500000,432000.000000,432000
1,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-71.19706999999998, -15.30859733333331)",75,75.0,215962.500000,208202.244850,416399.490567,424055
2,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -12.268649666666647)",180,180.0,215910.000000,203670.601273,407336.204630,419300
3,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -12.268649666666647)",160,160.0,215920.000000,203633.522222,407262.046296,419260
4,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-71.19706999999998, -12.268649666666647)",110,110.0,215945.000000,203606.745312,407208.491898,419230
5,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -9.228701999999984)",205,205.0,215897.500000,203627.506481,407250.015336,419255
6,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -9.228701999999984)",225,225.0,215887.500000,203565.737500,407126.477604,419190
7,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-78.76656799999994, -6.188754333333321)",210,210.0,215895.000000,199166.179919,398327.362269,414460
8,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-76.24340199999995, -6.188754333333321)",255,127.5,143915.000000,199018.337211,398031.677373,414450
9,5,5,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-73.72023599999997, -6.188754333333321)",275,137.5,143908.333333,138641.636863,277278.276910,335650


In [21]:
constellation = ctll.Ctll.from_sats(sat)

In [22]:
help(Camera)

Help on class Camera in module CtllDes.core.instrument:

class Camera(Instrument)
 |  Camera(f_l, s_w)
 |  
 |  Method resolution order:
 |      Camera
 |      Instrument
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, f_l, s_w)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  coverage(self, lons, lats, r, v, target, R)
 |      Coverage functions is associated with the coverage module.
 |      Any overwrited child method coverage must accept the specified parameters
 |      and return a list or iterable with 1 or 0, in view or not respectively.
 |      
 |      Parameters
 |      ----------
 |      lons : ~astropy.units.quantity.Quantity 
 |              array of longitudes as they come from the ssps method.
 |      lats : ~astropy.units.quantity.Quantity 
 |              array of latittudes as they come from the ssps method
 |      r : ~astropy.units.quantity.Quantity
 |              satellite's positions
 |      v : ~astro

In [23]:
from CtllDes.requests.coverage import symmetric_with_roll

#What does exactly symmetric_disk do?
help(symmetric_with_roll)



#A more realistic Instrument that uses one of the few coverage methods already written.

class RollCamera(Instrument):
    def __init__(self,FOV,roll_angle):
        """Constructor for RollCamera.
        
        Parameters
        ----------
        FOV : ~astropy.units.quantity.Quantity
            field of view, angle quantity
        roll_angle : ~astropy.units.quantity.Quantity
            maximum rolling angle
        """
        
        super().__init__()
        self.FOV = FOV.to(u.rad)
        self.roll = roll_angle.to(u.rad)
        
    def coverage(self, lons, lats, r, v, target, R):
        return symmetric_with_roll(self.FOV,
                                      lons,
                                      lats,
                                      r,
                                      v,
                                      target,
                                      R,
                                      roll_angle = self.roll)


Help on function symmetric_with_roll in module CtllDes.requests.coverage:

symmetric_with_roll(FOV, lons, lats, r, v, target, R, roll_angle=0)
    coverage method 
    
    This coverage method, is symmetric with the roll capabilities.
    It is just a potential coverage obtained by stipulating the new field of
    view, obtained by the roll angle in any direction. Perpendicular to velocity
    rolls are not taken into account since, increase in coverage from this 
    analysis are restricted to a few seconds of the satellite passing.



In [24]:
roll_cam = RollCamera(0.15*u.rad,15*u.deg)
sat.update_instruments(roll_cam, f=True)
sat.instruments[0]

In [ ]:
roll_cov = Coverages.from_sat(sat,tgts,0.4, dt=10, drag=False, J2=True)

In [50]:
rollcovdf = roll_cov.to_df()
rollcovdf 

,T,dt,Satellite ID,Target,accumulated,mean gap light,mean gap dark,response time,average time gap,max gap
0,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-70.70144810714284, -18.03939777966099)",70,70.0,17245.0,11158.585069,22307.190394,26630
1,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-70.47616542857142, -18.03939777966099)",60,60.0,17250.0,11160.862269,22311.741898,26630
2,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-70.25088274999999, -18.03939777966099)",60,60.0,17250.0,11160.862269,22311.741898,26630
3,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-70.02560007142856, -18.03939777966099)",60,60.0,17250.0,11155.436921,22300.891204,26620
4,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-70.92673078571427, -17.73025055932201)",80,80.0,17240.0,11156.310764,22302.644676,26630
...,...,...,...,...,...,...,...,...,...,...
1564,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-74.54679037931031, -0.7271534406779594)",340,340.0,17110.0,8478.750000,16947.598380,17430
1565,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-75.41684762068961, -0.4180062203389774)",350,350.0,17105.0,8473.527199,16937.155671,17410
1566,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-75.19933331034478, -0.4180062203389774)",340,340.0,17110.0,8478.567708,16947.233796,17420
1567,0.4,10,295b4aff-8541-48ba-8dbe-8c5b7b336322,"(-74.98181899999996, -0.4180062203389774)",350,350.0,17105.0,8473.706597,16937.514468,17420


In [55]:
roll_accum = rollcovdf['accumulated'].to_numpy(dtype=float)
roll_accum /= max(roll_accum)

response_time = rollcovdf['response time'].to_numpy()
response_time = 1/response_time
response_time -= min(response_time)
response_time /= max(response_time)

roll_avg = rollcovdf['average time gap'].to_numpy(dtype=float)
roll_avg = 1/roll_avg
roll_avg -= min(roll_avg)
roll_avg /= max(roll_avg) 

In [58]:
fig1 = plt.figure(figsize=(10,10))
ax1 = fig1.add_subplot(projection='3d')
ax1.plot_trisurf(target_lons, target_lats, roll_accum,
               antialiased=False,cmap='viridis')
ax1.set_title("Coverage over Perú")
ax1.set_xlabel("longitude [°]")
ax1.set_ylabel("latitude [°]")
ax1.set_zlabel("Accumulated time of coverage, normalized")

ax1.set_xlim(min(target_lons),max(target_lons))
ax1.set_ylim(min(target_lats),max(target_lats))
ax1.set_zlim(min(roll_accum),max(roll_accum))

ax1.scatter(target_lons,target_lats, np.zeros(len(target_lons)),
           s=100,c='k')


In [59]:
fig2 = plt.figure(figsize=(10,10))
ax2 = fig2.add_subplot(projection='3d')
ax2.plot_trisurf(target_lons, target_lats, response_time,
               antialiased=False,cmap='viridis')

ax2.set_title("Response time over Perú")
ax2.set_xlabel("longitude [°]")
ax2.set_ylabel("latitude [°]")
ax2.set_zlabel("1/tᵣ normalized")


ax2.set_xlim(min(target_lons),max(target_lons))
ax2.set_ylim(min(target_lats),max(target_lats))

ax2.scatter(target_lons,target_lats, np.zeros(len(target_lons)),
           s=100,c='k')

In [60]:
fig3 = plt.figure(figsize=(10,10))
ax3 = fig3.add_subplot(projection='3d')
ax3.plot_trisurf(target_lons, target_lats, roll_avg,
               antialiased=False,cmap='viridis')

ax3.set_title("Average time gap over Perú")
ax3.set_xlabel("longitude [°]")
ax3.set_ylabel("latitude [°]")
ax3.set_zlabel("Averaget time gap normalized")


ax3.set_xlim(min(target_lons),max(target_lons))
ax3.set_ylim(min(target_lats),max(target_lats))

ax3.scatter(target_lons,target_lats, np.zeros(len(target_lons)),
           s=100,c='k')

In [ ]:
plt.step